In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense,Flatten,Bidirectional,LSTM
from sklearn.preprocessing import label_binarize

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
emotions = np.array(['ang', 'exc', 'neu', 'sad'])
import pickle
with open('/home/ec22046/data.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [ ]:
layer_vanilla = tf.keras.layers.Attention()
x_train_speech = np.load('/home/ec22046/speech.npy')
print(x_train_speech.shape)
# read output label
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = label_binarize(y=Y,classes=emotions)

Y.shape

In [ ]:
speech_input = Input(shape=(100,34))
l1 = LSTM(512, return_sequences=True)(speech_input)
l1 = LSTM(256, return_sequences=True)(l1)
l2 = layer_vanilla([l1,l1])
l3 = Flatten()(l2)
l4 = Dense(512, activation='relu')(l3)
out_speech = Dense(128, activation='relu')(l4)
model_combined3 = Dense(4, activation='softmax')(out_speech)
model_combined = tf.keras.Model(speech_input, model_combined3)
model_combined.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
hist = model_combined.fit(x_train_speech, Y, batch_size=32, epochs=30, verbose=1, validation_split=0.2)

In [ ]:
model_combined.save('audio_lstm.h5')

In [ ]:
loaded_model =tf.keras.models.load_model('/home/ec22046/audio_lstm.h5')

In [ ]:
model = tf.keras.Model(inputs=loaded_model.inputs, outputs=loaded_model.layers[-2].output)

In [ ]:
model.summary()

In [ ]:
model.predict(x_train_speech)

In [ ]:
x_train_speech.shape[0]

In [ ]:
x_train_speech[0].reshape(1, 100, 34)

In [ ]:
audio_features = np.zeros((x_train_speech.shape[0],128))

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for i in tqdm(range(0,x_train_speech.shape[0])):
    x =model.predict(x_train_speech[i].reshape(1, 100, 34),verbose=0)
    audio_features[i] = x

In [ ]:
np.save('audio_features.npy',audio_features)

In [ ]:
audio_features.shape